RESOURCES:                                                                  \
https://python-overpy.readthedocs.io/en/latest/index.html                   \
https://wiki.openstreetmap.org/wiki/Main_Page                               \
https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_API_by_Example    \
https://www.openstreetmap.org/relation/7486148                              \
https://overpass-turbo.eu/    

In [22]:
import overpy
import pandas as pd

In [23]:
api = overpy.Overpass()

In [24]:
# Declare tags of interest
supermarket_tag = "'shop'='supermarket'"
convenience_tag = "'shop'='convenience'"
variety_store_tag = "'shop'='variety_store'"
general_store_tag = "'shop'='general'"
library_tag = "'amenity'='library'"
cafe_tag = "'amenity'='cafe'"
pub_tag = "'amenity'='pub'"
bar_tag = "'amenity'='bar'"
fast_food_tag = "'amenity'='fast_food'"
place_of_worship_tag = "'amenity'='place_of_worship'"
marketplace_tag = "'amenity'='marketplace'"  #e.g., farmer's market
bank_tag = "'amenity'='bank'"
pharmacy_tag = "'amenity'='pharmacy'"
ice_cream_tag = "'amenity'='ice_cream'"

In [25]:
# Construct area ID
relation_id = 7486148  #Guelph
OSM_base = 3600000000
area_id = str(OSM_base + relation_id)

In [26]:
# Run query 
key = 'amenity'
value = 'cafe'
tag = key +'='+ value

In [27]:
# A query function to get the Overpass API data
def osm_query(area_id,tag,element_type):

    result = api.query("""
        [out:json];
        area("""+area_id+""");
        ("""+element_type+"""["""+tag+"""](area);
        );
        out center;
        """)

    if element_type == "node":
        print("Nodes:", len(result.nodes))
    elif element_type == "way":
        print("Ways:", len(result.ways))
    elif element_type == "relation":
        print("Relations:", len(result.relations))

    return result

In [28]:
# Run queries and collate results 
data_dict = []

print("RESULTS:")

for element_type in ("node", "way", "relation"):
    query_result = osm_query(area_id, tag, element_type)

    if element_type == "node":
        for element in query_result.nodes:
            data_dict.append({
                        'ID': element.id,
                        'Name': element.tags.get("name", ""), 
                        'Value': element.tags.get(key, ""), 
                        'Latitude': element.lat,
                        'Longitude': element.lon
                    })

    elif element_type == "way":
        for element in query_result.ways:
            data_dict.append({
                        'ID': element.id,
                        'Name': element.tags.get("name", ""), 
                        'Value': element.tags.get(key, ""), 
                        'Latitude': element.center_lat,
                        'Longitude': element.center_lon
                    })

    elif element_type == "relation":
        for element in query_result.relations:
            data_dict.append({
                        'ID': element.id,
                        'Name': element.tags.get("name", ""), 
                        'Value': element.tags.get(key, ""), 
                        'Latitude': element.center_lat,
                        'Longitude': element.center_lon
                    })

result_db = pd.DataFrame(data_dict)

RESULTS:
Nodes: 51
Ways: 8
Relations: 0


In [29]:
# Examine results 
result_db.head(10)

,ID,Name,Value,Latitude,Longitude
0,825311398,Tim Hortons,cafe,43.5417689,-80.2445805
1,825311413,Atmosphere Cafe,cafe,43.5439483,-80.2488932
2,1357922183,Planet Bean Coffee,cafe,43.5323034,-80.2330668
3,1395246593,Red Brick Cafe,cafe,43.5465908,-80.2490199
4,1732487985,Williams Fresh Cafe,cafe,43.5218405,-80.2347807
5,1783614478,Tim Hortons,cafe,43.5776808,-80.2604727
6,2266364283,Planet Bean,cafe,43.5103628,-80.1965153
7,2425945218,Tim Hortons,cafe,43.5449260,-80.3061473
8,3347131750,Tim Hortons,cafe,43.5440623,-80.3076850
9,3631895695,Planet Bean Coffee,cafe,43.5580243,-80.2296869


In [30]:
# Output to csv
result_db.to_csv('result_db.csv',index=False)